In [1]:
import csv
import time
import numpy as np
np.random.seed(1)

In [2]:
from scipy.interpolate import interp1d

from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Reshape
from keras.layers import LSTM

Using Theano backend.


In [3]:
def read_data(filename):
    data = {}
    with open(filename, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in reader:
            block = row[-1][1:]
            if block.isdigit():
                block = int(block)
            del row[-1]
            del row[0]
            if not block in data:
                data[block] = {}
                data[block]['gyr'] = {}
                data[block]['acc'] = {}
                data[block]['gyr']['x'] = []
                data[block]['gyr']['y'] = []
                data[block]['gyr']['z'] = []
                data[block]['acc']['x'] = []
                data[block]['acc']['y'] = []
                data[block]['acc']['z'] = []
            frow = np.array(map(float, row))
            data[block]['acc']['x'].append(np.array(frow[range(0,len(row),6)]))
            data[block]['acc']['y'].append(np.array(frow[range(1,len(row),6)]))
            data[block]['acc']['z'].append(np.array(frow[range(2,len(row),6)]))
            data[block]['gyr']['x'].append(np.array(frow[range(3,len(row),6)]))
            data[block]['gyr']['y'].append(np.array(frow[range(4,len(row),6)]))
            data[block]['gyr']['z'].append(np.array(frow[range(5,len(row),6)]))
    return data

In [4]:
def inter(obs, ndots=100):
    obs = np.array(obs)
    if ndots == -1:
        return obs
    interpolated = []
    x = np.linspace(0, 100, num=obs.shape[0], endpoint=True)
    f = interp1d(x, obs, kind='slinear')
    xnew = np.linspace(0, 100, num=ndots, endpoint=True)
    interpolated.append(f(xnew))
    return interpolated

In [5]:
def array_from_dict(data, blocks=range(1, 16), ndots=-1, gyro=True, acc=True):
    X = []
    y = []
    for block in data:
        if block in blocks:
            for ind in range(len(data[block]['acc']['x'])):
                y.append(block)
                if gyro and acc:
                    X.append(np.dstack((inter(data[block]['acc']['x'][ind], ndots), 
                                        inter(data[block]['acc']['y'][ind], ndots), 
                                        inter(data[block]['acc']['z'][ind], ndots), 
                                        inter(data[block]['gyr']['x'][ind], ndots), 
                                        inter(data[block]['gyr']['y'][ind], ndots), 
                                        inter(data[block]['gyr']['z'][ind], ndots))))
                if (not gyro) and acc:
                    X.append(np.dstack((inter(data[block]['acc']['x'][ind], ndots), 
                                        inter(data[block]['acc']['y'][ind], ndots), 
                                        inter(data[block]['acc']['z'][ind], ndots))))
                if gyro and (not acc):
                    X.append(np.dstack((inter(data[block]['gyr']['x'][ind], ndots), 
                                        inter(data[block]['gyr']['y'][ind], ndots), 
                                        inter(data[block]['gyr']['z'][ind], ndots))))
    return X, np.array(y)

### Start from here

In [6]:
def lstm(X, y, n_units=None, n_features=None, n_epoch=None):
    b = np.zeros((len(y), 16))
    b[np.arange(len(y)), y-1] = 1
    y = b  # One hot encoded y
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    model = Sequential()  # Creates a model
    model.add(LSTM(n_units, input_dim=n_features)) 
    model.add(Dense(16, activation='sigmoid'))  # Kind of output layer
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    #print(model.summary())
    for seq, label in zip(X_train, y_train):
        label = label.reshape((1, -1))
        model.fit(seq, label, nb_epoch=n_epoch, batch_size=1, verbose=0)

    y_pred = []
    y_true = []
    scores = []
    for seq, label in zip(X_test, y_test):
        label = label.reshape((1, -1))
        scores.append(model.evaluate(seq, label, verbose=0)[1])
        y_pred.append(model.predict_classes(seq, verbose=0)[0])
        y_true.append(np.where(label == 1)[1][0])
    return np.mean(scores), confusion_matrix(y_true, y_pred)

In [7]:
data1 = read_data('data1.csv')  # Your data
data2 = read_data('data2.csv')  # My data (I dont have a gyroscope)

1. Can you create a confusion matrix (error matrix) so that we could see if there are certain classes which are typically confused, or the recognition is poor in general.

In [8]:
X, y = array_from_dict(data1, ndots=-1)

In [9]:
scores = []
matrices = []
for i in range(10):
    s, m = lstm(X, y, n_units=120, n_features=6, n_epoch=20)
    scores.append(s)
    matrices.append(m)
print 'score: ', np.max(scores)
print 'matrix: '
print matrices[np.where(np.array(scores) == np.max(scores))[0][0]]

score:  0.695652173913
matrix: 
[[1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 1 0 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 0 0 5]]


3. can you select two very different classes (like 1 vs 9 or 11 vs 13) and test how the RNN can learn these two classes? In this case, please use much less hidden units, I think 3-10 would be enough.

In [10]:
X, y = array_from_dict(data1, blocks=[1, 9], ndots=-1) # 1 vs. 9

In [11]:
scores = []
matrices = []
for i in range(10):
    s, m = lstm(X, y, n_units=10, n_features=6, n_epoch=20) # 10 units
    scores.append(s)
    matrices.append(m)
print 'score: ', np.max(scores)
print 'matrix: '
print matrices[np.where(np.array(scores) == np.max(scores))[0][0]]

score:  0.2
matrix: 
[[1 0]
 [4 0]]


In [12]:
scores = []
matrices = []
for i in range(10):
    s, m = lstm(X, y, n_units=40, n_features=6, n_epoch=20) # 40 units
    scores.append(s)
    matrices.append(m)
print 'score: ', np.max(scores)
print 'matrix: '
print matrices[np.where(np.array(scores) == np.max(scores))[0][0]]

score:  1.0
matrix: 
[[1 0]
 [0 4]]


In [13]:
X, y = array_from_dict(data1, blocks=[11, 13], ndots=-1) # 11 vs. 13

In [14]:
scores = []
matrices = []
for i in range(10):
    s, m = lstm(X, y, n_units=10, n_features=6, n_epoch=20) # 10 units
    scores.append(s)
    matrices.append(m)
print 'score: ', np.max(scores)
print 'matrix: '
print matrices[np.where(np.array(scores) == np.max(scores))[0][0]]

score:  1.0
matrix: 
[[2 0]
 [0 2]]


In [15]:
scores = []
matrices = []
for i in range(10):
    s, m = lstm(X, y, n_units=40, n_features=6, n_epoch=20) # 40 units
    scores.append(s)
    matrices.append(m)
print 'score: ', np.max(scores)
print 'matrix: '
print matrices[np.where(np.array(scores) == np.max(scores))[0][0]]

score:  1.0
matrix: 
[[2 0]
 [0 2]]


4. can you  reduce the sampling rate? Maybe there is a very little difference in data points at subsequent time positions like t and t+1. Would you invesitgate the performance when only every 10th or 20th data point is kept in the time seris?

In [16]:
X, y = array_from_dict(data1, ndots=10) # each sequence is interpoleted to 10 points

In [17]:
scores = []
matrices = []
for i in range(10):
    s, m = lstm(X, y, n_units=120, n_features=6, n_epoch=20)
    scores.append(s)
    matrices.append(m)
print 'score: ', np.max(scores)
print 'matrix: '
print matrices[np.where(np.array(scores) == np.max(scores))[0][0]]

score:  0.521739130435
matrix: 
[[1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 0 0]
 [3 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 2 0 0 0 0 0 0 0 3]]


5. Can you try RNN on your data (i.e. which does not have gyroscope data). Likewise, can you try using only acc or only gyro data from my data?
 

In [18]:
X, y = array_from_dict(data1, ndots=-1, gyro=False) # your data; acceleration only

In [19]:
scores = []
matrices = []
for i in range(10):
    s, m = lstm(X, y, n_units=120, n_features=3, n_epoch=20)
    scores.append(s)
    matrices.append(m)
print 'score: ', np.max(scores)
print 'matrix: '
print matrices[np.where(np.array(scores) == np.max(scores))[0][0]]

score:  0.652173913043
matrix: 
[[1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 2 0 0]
 [0 0 0 0 0 0 0 0 0 3 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 4]]


In [20]:
X, y = array_from_dict(data1, ndots=-1, acc=False) # your data; gyroscope only

In [21]:
scores = []
matrices = []
for i in range(10):
    s, m = lstm(X, y, n_units=120, n_features=3, n_epoch=20)
    scores.append(s)
    matrices.append(m)
print 'score: ', np.max(scores)
print 'matrix: '
print matrices[np.where(np.array(scores) == np.max(scores))[0][0]]

score:  0.478260869565
matrix: 
[[1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 2 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 0 0 2 0 0]
 [4 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 5]]


In [22]:
X, y = array_from_dict(data2, ndots=-1, gyro=False) # my data; acceleration only

In [23]:
scores = []
matrices = []
for i in range(10):
    s, m = lstm(X, y, n_units=120, n_features=3, n_epoch=20)
    scores.append(s)
    matrices.append(m)
print 'score: ', np.max(scores)
print 'matrix: '
print matrices[np.where(np.array(scores) == np.max(scores))[0][0]]

score:  0.451612903226
matrix: 
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 2 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 2]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 2 0 0 0 3 0 2 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 0 0 0 3 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
